## Imports

In [21]:
import sys
import pandas as pd
import numpy as np
import pickle

# Import my libs
sys.path.append('../')

from src.features.balance import BalanceMixin

modelisationTuple

In [22]:
with open('../data/processed/modelling_tuple', 'rb') as f:
    M = pickle.load(f)

## SmokeTest

#### BalanceMixin

In [23]:
print("Taille du jeu de données d'entrainement : ", M.train_X.shape[0])

Taille du jeu de données d'entrainement :  217877


In [24]:
# prevalence
y = M.train_y
out = pd.concat([
    np.round(y.value_counts(normalize=True, ascending=False).rename('normalized'), 4) * 100,
    y.value_counts(normalize=False, ascending=False).rename('number'),
], axis=1)
out

,normalized,number
Class,,
0.0,99.83,217497
1.0,0.17,380


In [25]:
balance_instance = BalanceMixin()

In [28]:
x, y = balance_instance._balance(M.train_X, M.train_y)

y = pd.Series(y)
# new prevalence
out = pd.concat([
    np.round(y.value_counts(normalize=True, ascending=False).rename('normalized'), 4) * 100,
    y.value_counts(normalize=False, ascending=False).rename('number'),
], axis=1)
out

,normalized,number
0.0,50.0,380
1.0,50.0,380
